In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/config.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/tokenizer.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/metadata.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/model.weights.h5
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/assets/tokenizer/vocabulary.txt
/kaggle/input/palm-essay-gen-itr1/Palm_gen_essay_1.csv
/kaggle/input/roberta/tensorflow2/en-cased-preprocess/1/saved_model.pb
/kaggle/input/roberta/tensorflow2/en-cased-preprocess/1/keras_metadata.pb
/kaggle/input/roberta/tensorflow2/en-cased-preprocess/1/variables/variables.index
/kaggle/input/roberta/tensorflow2/en-cased-preprocess/1/variables/variabl

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
snowball = SnowballStemmer(language='english')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as XGB
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
from numba import cuda
from nltk.tokenize import word_tokenize
import tensorflow_text as tftext
import tensorflow_hub as tfhub

In [3]:
train_csv = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_csv

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [4]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [5]:
palm_gen_essay =pd.read_csv('/kaggle/input/palm-generated-2/Palm_gen_essay_2.csv')
palm_gen_essay.head()

,id,prompt_id,text,generated
0,0,0.0,"In the United States, we are a car-dependent s...",1.0
1,1,0.0,"In recent years, there has been a growing tren...",1.0
2,2,0.0,Limiting car usage has many advantages. It can...,1.0
3,3,0.0,"In recent years, there has been a growing move...",1.0
4,4,0.0,Limiting car usage has many advantages. It can...,1.0


In [6]:
palm_gen_essay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1255 non-null   int64  
 1   prompt_id  1255 non-null   float64
 2   text       1255 non-null   object 
 3   generated  1255 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 39.3+ KB


In [7]:
palm_gen_essay['generated'] = palm_gen_essay['generated'].astype('int64')

In [8]:
# Concatenate on specific columns (e.g., 'A' and 'B')
data = pd.concat([train_csv[['text','generated']], palm_gen_essay[['text','generated']]], axis=0, join='outer')
data.shape


(2633, 2)

In [9]:
data.head()

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [10]:
# define a text embedding model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = tfhub.KerasLayer("https://kaggle.com/models/kaggle/roberta/frameworks/TensorFlow2/variations/en-cased-preprocess/versions/1")
encoder_inputs = preprocessor(text_input)

encoder = tfhub.KerasLayer("https://www.kaggle.com/models/kaggle/roberta/frameworks/TensorFlow2/variations/en-cased-l-12-h-768-a-12/versions/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      
sequence_output = outputs["sequence_output"] 

dense_1 = tf.keras.layers.Dense(128 , activation='relu')(pooled_output)
dropout = tf.keras.layers.Dropout(0.7 , name="dropout1")(pooled_output)
dense_2 = tf.keras.layers.Dense(64 , activation='relu')(dropout)
dropout1 = tf.keras.layers.Dropout(0.5 , name="dropout2")(dense_2)
dense_3 = tf.keras.layers.Dense(16,activation='relu')(dropout1)

dense_out = tf.keras.layers.Dense(1 , activation='sigmoid', name='output')(dropout1)


model_2 = tf.keras.Model(inputs=text_input, outputs=dense_out)
model_2.summary()

Attaching model 'kaggle/roberta/tensorflow2/en-cased-preprocess/1' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Attaching model 'kaggle/roberta/tensorflow2/en-cased-l-12-h-768-a-12/1' to your Kaggle notebook...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['input_1[0][0]']             
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [11]:
def cleantext(data) :
    stemtext = []
    len_text = []
    para = data['text'].tolist()
    for paragraph in para:
        char = [char for char in paragraph if char not in string.punctuation]
        word = "".join(char).split(" ")
        words = [word.lower() for word in word if word not in stopwords.words('english')]
        stemwords = [SnowballStemmer('english').stem(word) for word in words]
        len_text.append(len(stemwords))
        stemtext.append(" ".join(stemwords))
    return stemtext , len_text

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [13]:
data['text'] = cleantext(data)[0]

In [14]:
data.head()

,text,generated
0,car car around sinc becam famous 1900s henri f...,0
1,transport larg necess countri worldwid with do...,0
2,america love affair vehicl seem cool say elisa...,0
3,how often ride car do drive one motor vehicl w...,0
4,car wonder thing they perhap one world greates...,0


In [15]:
x_train, x_test ,y_train , y_test = train_test_split(data['text'], data['generated'],test_size=0.2)


In [16]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.00005),
              loss='binary_crossentropy',
              metrics=["accuracy"])
metric = 'val_accuracy'
checkpoint_filepath = 'checkpoint.hdf5'
callback_list = [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                    monitor=metric,
                                                    verbose=2,
                                                    save_best_only=True,
                                                    mode='max'), 
                 tf.keras.callbacks.EarlyStopping(monitor=metric,
                                                  patience=0,
                                                  restore_best_weights=True)
                ]
history = model_2.fit(x_train, y_train , batch_size=32,
                    epochs=1 , validation_data=(x_test, y_test))
model_2.save("bertmodel_2_v2")

66/66 [==============================] - 1666s 25s/step - loss: 0.3323 - accuracy: 0.8466 - val_loss: 0.0438 - val_accuracy: 1.0000


In [17]:
final_test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
final_test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [18]:
test_text = cleantext(final_test)[0]
final_test['text'] = test_text

In [19]:
pred = model_2.predict(final_test['text'].values)
pred

1/1 [==============================] - 3s 3s/step


array([[0.11172061],
       [0.09365334],
       [0.06928797]], dtype=float32)

In [20]:
test_result = pred[:,0]
test_result

array([0.11172061, 0.09365334, 0.06928797], dtype=float32)

In [21]:
final_submission = pd.DataFrame(final_test['id'])
final_submission['generated'] = test_result
final_submission

,id,generated
0,0000aaaa,0.111721
1,1111bbbb,0.093653
2,2222cccc,0.069288


In [22]:
final_submission.to_csv('submission.csv', index=False)